# Домашнее задание по рекомендательным системам

В данном домашнем задании вам предлагается реализовать User-based рекомендательную систему. Так же требуется реализовать несколько вспомогательных функций, шаблоны которых вы можете найти в `utils.py`.

Требования к выполнению задания:
- Реализация функции из `utils.py` засчитывается, только если пройдены все соответствующие тесты из `test.py`. Запуск тестов: <font color='red'>pytest test.py</font>. Для тестов вам потребуются библиотеки `numpy`, `scipy`, `pytest` и `hypothesis`.
- Плагиат запрещен. Если будет замечено, что часть задания списана, то 0 баллов ставится как списывающему, так и давшему списать.
- Если пользуетесь кодом из открытых источников, то указывайте ссылки, откуда взяли решение. Иначе такой код может быть воспринят как плагиат.
- При выполнении задания нельзя использовать библиотеку `scipy` и функцию `numpy.linalg.norm`

При запуске тестов могут появиться предупреждения: PearsonRConstantInputWarning и PearsonRNearConstantInputWarning. На них можно не обращать внимания.

Возможный максимум баллов за задание: 10 баллов <br>
Дедлайн: ??? <br>
Штраф: ??? - будет ли в курсе штраф? <br>
<br>
Для ускорения проверки, напишите здесь получившееся количество баллов: ...

In [3]:
from utils import euclidean_distance, euclidean_similarity, pearson_similarity, apk, mapk
import numpy as np

## 1. Метрика сходства
<b>1.1. Реализация метрик (2 балла)</b>

Первое, с чем необходимо разобраться, при реализации User-based подхода, это с метрикой, с помощью которой будет решаться, насколько похожи пользователи. Вам предлагается реализовать 2 метрики: на основе евклидовой метрики и коэффициент корреляции Пирсона. Шаблоны для обоих функций можете найти в `utils.py`. Не забудьте проверить реализацию на тестах.

Евклидова метрика:
\begin{equation}
d(p,q)=\sqrt{(p_1-q_1)^2+(p_2-q_2)^2+\dots+(p_n-q_n)^2} = \sqrt{\sum_{k=1}^n (p_k-q_k)^2}
\end{equation}

В этом случае $d(p, q) \in [0, \infty)$, при этом если $d(p, q) \to 0$, то $sim(p, q) \to 1$. С учетом этого конечная формула будет выглядеть следующим образом:
\begin{equation}
sim(p, q) = \frac{1}{1 + d(p, q)}
\end{equation}
Так же в этой формуле не будет проблем с делением на 0.

Коэффициент корреляции Пирсона:
\begin{equation}
r_{xy} = \frac {\sum_{i=1}^{m} \left( x_i-\bar{x} \right)\left( y_i-\bar{y} \right)}{\sqrt{\sum_{i=1}^{m} \left( x_i-\bar{x} \right)^2 \sum_{i=1}^{m} \left( y_i-\bar{y} \right)^2}}
\end{equation}

In [9]:
!pytest test.py

============================= test session starts =============================
platform win32 -- Python 3.8.8, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
rootdir: d:\Google\Projects\ml_tinkoff_2021\notebooks\lesson_10_recommend\hw
plugins: anyio-2.2.0, hypothesis-6.34.2
collected 11 items

test.py .....FFFFFF                                                      [100%]

================================== FAILURES ===================================
___________________ test_apk[predicted0-0.3333333333333333] ___________________

predicted = [1, 4, 5], expected_score = 0.3333333333333333

    @pytest.mark.parametrize(
        'predicted, expected_score',
        (
            ([1, 4, 5], 0.3333333333333333),
            ([4, 1, 5], 0.16666666666666666),
            ([5, 4, 1], 0.1111111111111111),
            ([4, 1, 2], 0.38888888888888884),
            ([3, 2, 1], 1.0),
        )
    )
    def test_apk(predicted, expected_score):
        actual = np.array([1, 2, 3])
>       np.testing.asse

<b>1.2. (1 балл)</b>

Рассмотрим пользователей $u$ и $v$. Им соотвествуют векторы $x_u$ и $x_v$, где $x_u[i] = r_{ui}$ и $x_v[i] = r_{vi}$. Из лекции известно, что похожесть между векторами $x_u$ и $x_v$ вычисляются только для тех индексов i, для которых существует и $r_{ui}$, и $r_{vi}$. То есть верно следуюющее:
\begin{equation}
sim(u, v) = sim(x_uI_{uv}, x_vI_{uv}),
\end{equation}
где $I_{uv} = [i | \exists r_{ui} \& \exists r_{vi}]$. При этом если $I_{uv} = \emptyset$, то $sim(u, v) \to -\infty$.

Реализуйте два новых метода, которые переиспользуют написанные вами `euclidean_distance` и `pearson_distance`, добавляющие условия на $x_u$ и $x_v$. Считается, что $x_u[i] = 0$, если $\nexists r_{ui}$. То же верно для $x_v$.

При реализации заданий можно как написать новые функции, так и использовать декораторы.

In [11]:
import numpy as np
from utils import euclidean_similarity, pearson_similarity

In [20]:
def intersection_items(f):
    def decorator(x: np.array, y: np.array):
        indices = np.arange(len(x))
        intresec_idx = indices[x != 0 & y != 0]
        return f(x[intresec_idx], y[intresec_idx])
    return decorator

In [22]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
@intersection_items
def euclidean_similarity(x: np.array, y: np.array) -> float:
    """
    Calculate euclidean similarity between points x and y
    Args:
        x, y: two points in Euclidean n-space
    Returns:
        Similarity between points x and y
    """
    return 1 / (1 + euclidean_distance(x, y))

@intersection_items
def pearson_similarity(x: np.array, y: np.array) -> float:
    """
    Calculate a Pearson correlation coefficient given 1-D data arrays x and y
    Args:
        x, y: two points in n-space
    Returns:
        Pearson correlation between x and y
    """
    f = lambda z: np.sum((z - z.mean()) ** 2)

    return np.sum((x - x.mean()) * (y - y.mean())) / np.sqrt(f(x) * f(y))

In [23]:
!pytest test.py

============================= test session starts =============================
platform win32 -- Python 3.8.8, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
rootdir: d:\Google\Projects\ml_tinkoff_2021\notebooks\lesson_10_recommend\hw
plugins: anyio-2.2.0, hypothesis-6.34.2
collected 11 items

test.py .....FFFFFF                                                      [100%]

================================== FAILURES ===================================
___________________ test_apk[predicted0-0.3333333333333333] ___________________

predicted = [1, 4, 5], expected_score = 0.3333333333333333

    @pytest.mark.parametrize(
        'predicted, expected_score',
        (
            ([1, 4, 5], 0.3333333333333333),
            ([4, 1, 5], 0.16666666666666666),
            ([5, 4, 1], 0.1111111111111111),
            ([4, 1, 2], 0.38888888888888884),
            ([3, 2, 1], 1.0),
        )
    )
    def test_apk(predicted, expected_score):
        actual = np.array([1, 2, 3])
>       np.testing.asse

## 2. User-based method
<b>2.1. (3 балла)</b> 

Реализовать User-based подход, реализовав методы класса `UserBasedRecommendation`, основанного на использовании `NearestNeighbors`. В качестве метрики может для нахождения похожих пользователей может быть использована как евклидова метрика, так и коэффициент корреляции Пирсона.

Не забывайте, что `NearestNeighbors` ищет минимум расстояния между элементами, поэтому логично в качестве метрики при инициализации `NearestNeighbors` использовать обратную метрике схожести. То есть такую, что когда $sim(u, v) \to 1$, то $d(u, v) \to 0$. Например: $d(u, v) = 1 - sim(u, v)$

In [65]:
k = np.array([[2, 4, 1, 3], [2, 6, 4, 5], [5, 4, 2, 3]])
indices = np.arange(4)
np.argsort(k[1])[k[1, np.argsort(k[1])] > 2][::-1]

array([1, 3, 2], dtype=int64)

In [66]:
k[0, np.argsort(k[0])[::-1]] > 2

array([ True,  True, False, False])

In [67]:
from sklearn.neighbors import NearestNeighbors
neighbors = NearestNeighbors(metric= )
neighbors.fit()

In [80]:
from sklearn.neighbors import NearestNeighbors
from typing import Optional

from collections import Counter


class UserBasedRecommendation:
    def __init__(self, metric: str = 'euclidean', n_recommendations: int = 5, alpha: float = 0.8):
        """
        Args:
            metric: name of metric: ['euclidean', 'pearson']
            n_recommendations: number of recommendations. Also can be specified self.make_recommendation
            alpha: similarity threshold: if sim(u, v) > alpha then u and v are similar
        """
        # your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
        self.metric = self.define_metric(metric)
        self.n_recommend = n_recommendations
        self.threshold = alpha

    def define_metric(self, metric: str):
        if metric == 'euclidean':
            return euclidean_similarity
        elif metric == 'pearson':
            return pearson_similarity
        else:
            ValueError("You can use metrics 'euclidean' or 'pearson'")

    def fit(self, X: np.array):
        """
        Args:
            X: matrix N x M where X[u, i] = r_{ui} if r_{ui} exists else X[u, i] = 0
        """
        # your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
        # self.users_dist = 1 - self.sim_matrix(X)
        self.X = X
        self.neighbors = NearestNeighbors(metric=self.distance, radius=self.threshold)
        self.neighbors.fit(self.X)

    def __find_closest_users(self, user_id: int, n_closest_users: int):
        # argsort_users = np.argsort(self.users_dist[user_id])
        # idx_user_sim = argsort_users[self.users_dist[user_id, argsort_users] < (1 - self.threshold)]
        # return idx_user_sim[:n_closest_users]
        return self.neighbors.kneighbors(X=[self.X[user_id]], n_neighbors=n_closest_users)

    def make_recommendation(self, user_id: int, n_recommendations: Optional[int] = None):
        """
        Args:
            user_id: user id to whom you want to recommend
            n_recommendations: number of recommendations
        """
        # your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
        if n_recommendations is not None:
            self.n_recommend = n_recommendations
        users_closer = self.__find_closest_users(user_id, 3)
        return np.argsort(self.x[users_closer, :].sum(1))[:self.n_recommend]

    # def sim_matrix(self, x: np.array) -> np.array:
    #     """
    #     Calculate similarity matrix with metric
    #     Args:
    #         x: matrix N x M where N - number of users, M - number of items
    #     Return:
    #         similarities: similarity matrix N x N
    #     """
    #     m = x.shape[0]
    #     similarities = [[self.metric(x[i,:], x[j,:]) for j in range(m)] for i in range(m)]
    #     return similarities

    def distance(self, x: np.array, y: np.array):
        return 1 - self.metric(x, y)

@intersection_items
def euclidean_similarity(x: np.array, y: np.array) -> float:
    """
    Calculate euclidean similarity between points x and y
    Args:
        x, y: two points in Euclidean n-space
    Returns:
        Similarity between points x and y
    """
    return 1 / (1 + euclidean_distance(x, y))

@intersection_items
def pearson_similarity(x: np.array, y: np.array) -> float:
    """
    Calculate a Pearson correlation coefficient given 1-D data arrays x and y
    Args:
        x, y: two points in n-space
    Returns:
        Pearson correlation between x and y
    """
    f = lambda z: np.sum((z - z.mean()) ** 2)

    return np.sum((x - x.mean()) * (y - y.mean())) / np.sqrt(f(x) * f(y))

<b>2.2. (1 балла)</b>

Приведите пример, для которого использование разных метрик будет давать разные рекомендации. Объясните свой пример.

In [81]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
X = np.array([
    [8, 5, 4, 7, 1],
    [7, 1, 3, 8, 3],
    [8, 2, 6, 9, 3],
    [9, 6, 8, 7, 5]
]).T
recommend = UserBasedRecommendation(metric='euclidean')
recommend.fit(X)
recommend.make_recommendation(user_id=1)

TypeError: ufunc 'bitwise_and' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

<b>Объяснение:</b> Для ясности пронумеруем товары с 0. Видно, что третьему пользователю так же как и второму больше понравился товар 1 и чуть меньше понравился товар 2, тогда как первому пользователю оба эти товары не сильно понравились, причем товар 1 ему понравился меньше, чем товар 2, то есть наблюдается обратная завизимость. Но все эти факторы никак не учитываются при использовании евклидовой метрики, поэтому в первом случае алгоритм посчитал, что первый и третий пользователь похожи только по тому, что они ставят оценки примерно в одном и том отрезке: {0, 1}, что является довольно странным предположением. А алгоритм, использующий коэффициент корреляции Пирсона учитывает эти факторы, поэтому находит третьему пользователю соседа со схожими интересами в виде второго пользователя. Отсюда получаем разные рекомендации.

## 3. Оценка качества
<b>3.1. (1 балл)</b>

Реализуйте Average Precision at k и Mean Average Precision at k. Шаблоны можете найти в `utils.py`.
\begin{align*}
AP@K = \frac{1}{m}\sum_{k=1}^K P(k)*rel(k), \\
MAP@K = \frac{1}{|U|}\sum_{u=1}^{|U|}(AP@K)_u
\end{align*}
где $P(k)$ - Precision at k, $rel(k) = 1$, если рекомендация релевантна, иначе $rel(k) = 0$.

## 4. Применение модели
<b>4.1. (2 балла)</b>

Выгрузите датасет `ratings_small.csv`: https://www.kaggle.com/rounakbanik/the-movies-dataset#ratings_small.csv

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('ratings_small.csv', index_col=False)
data.shape

In [ ]:
data.userId.min(), data.userId.max(), len(data.userId.unique())

In [ ]:
data.movieId.min(), data.movieId.max(), len(data.movieId.unique())

Для простоты работы с данными, измените нумерацию пользователей и фильмов так, чтобы нумерация начиналась с 0 и шла непрерывно.

In [ ]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
pass

In [ ]:
data.userId.min(), data.userId.max(), len(data.userId.unique())

In [ ]:
data.movieId.min(), data.movieId.max(), len(data.movieId.unique())

Удалим для наиболее активных пользователей 5 оценок

In [ ]:
active_users = data.userId.value_counts()[:10].index
test_data = pd.DataFrame([], columns=data.columns)
for user_id in active_users:
    _, test = train_test_split(data[data.userId == user_id], test_size=5, random_state=42)
    test_data = test_data.append(test, ignore_index=True)
    data = data[~((data.userId == user_id) & (data.movieId.isin(test.movieId.values)))]
data.shape, test_data.shape

Преобразуем данные в таблицу `X`, с которой может работать `UserBasedRecommendation`, где $X_{ui} = r_{ui}$, если пользователь $u$ поставил оценку фильму $i$, и $X_{ui} = 0$, если пользователь $u$ не проставил оценку фильму $i$.

Вам может пригодиться `csr_matrix`.

In [ ]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
pass

Для пользователей, у которых были удалены фильмы, найдите топ 100 фильмов, который должен посмотреть каждый из этих пользователей, используя `UserBasedRecommendation`. Не забудьте подобрать параметр alpha.

In [ ]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
pass

Используя метрику `MAP@5`, `MAP@10` и `MAP@100`, определите, насколько эффективна user-based рекомендательная система для данной задачи.

In [ ]:
from utils import mapk

In [ ]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
pass

Как можно улучшить работу модели?

<b>Ответ:</b>...